In [26]:
import pickle
from os.path import exists
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import bitstring
import cv2 
from glob import glob

In [27]:
# Proposed change of code by Ferhat Özgur Catak. 
# The change is in the for loop with a defined start and end to the loop with a maximum iterations of 50.
# Based on that shapes are between 3 and 185472
files = glob('../GeneratedData\\NR*.png')
num_of_files = len(files)

# def get_entropy_spectrum(c, k, sel_img_idx):
def get_entropy_spectrum(c, k, sel_img_idx, rng):
    sel_img = files[sel_img_idx]
    img = cv2.imread(sel_img)
    start_x = rng.integers(img.shape[0] - c)
    start_y = rng.integers(img.shape[1] - k)


    selected_spectogram = img[start_x:start_x+c, start_y:start_y+k, :].flatten()
    start = (selected_spectogram.shape[0] % 3) + 1
    end = selected_spectogram.shape[0] % 51 
    # random_bits = bitstring.BitArray(float=selected_spectogram[0], length=32)
    random_bits = bitstring.BitArray(uint=int(selected_spectogram[0]), length=8)
    
    for i in range(start, end):
        # tmp = bitstring.BitArray(float=selected_spectogram[i], length=32) 
        tmp = bitstring.BitArray(uint=int(selected_spectogram[i]), length=8) 
        random_bits = random_bits ^ tmp # xor
        random_bits = random_bits ^ random_bits << 13
        random_bits = random_bits ^ random_bits >> 17
        random_bits = random_bits ^ random_bits << 5
    
    return random_bits.uintle, random_bits

In [28]:
# Creating regular dataset with 1 000 000 entropies.
entropies = []
for i in tqdm(range(0,1000000)):
    rng = np.random.default_rng(123+i)
    c, k, s = rng.integers(1,369), rng.integers(1,169), rng.integers(0,2015)
    entropies.append(get_entropy_spectrum(c,k,s,rng))

100%|██████████| 1000000/1000000 [23:11<00:00, 718.91it/s] 


In [29]:
with open("entropies_nochaot_8bit.bin", "wb") as f:
    for entropy in entropies:
        f.write(entropy[0].to_bytes(1, byteorder='big', signed=False))
        # f.write(entropy[0].to_bytes(4, byteorder='big', signed=False))

In [30]:
# Creating restart dataset with 1 000 000 entropies total
restart = []
for i in tqdm(range(0,1000)):
    rng = np.random.default_rng(123+i)
    temp = []
    for j in range(0,1000):
        c, k, s = rng.integers(1,369), rng.integers(1,169), rng.integers(0,2015)
        temp.append(get_entropy_spectrum(c,k,s,rng))
    restart.append(temp)

100%|██████████| 1000/1000 [21:36<00:00,  1.30s/it]


In [31]:
with open("restart_nochaot_8bit.bin", "wb") as f:
    for r in restart:
        # temp = []
        for entropy in r:
            # temp.append(entropy[0].to_bytes(4, byteorder='big', signed=False))
            f.write(entropy[0].to_bytes(1, byteorder='big', signed=False))
            # f.write(entropy[0].to_bytes(4, byteorder='big', signed=False))
        # f.write(temp)

Tried running with list comprehensions and without to see which is faster since it takes a very long time to run.  
List comprehensions with 10: 1m 15.7s   
Regular loop with 10: 1m 0.8s  
List comprehensions with 20: 18m 9.8s  
Regular loop with 20: 18m 18.4s  

In [46]:
rng = np.random.default_rng(12345)
for i in range(20):
    # print(get_entropy_spectrum(1,1,i,rng))
    print(get_entropy_spectrum(368,168,i,rng))

(4143306973, BitArray('0xddd8f5f6'))
(955241019, BitArray('0x3bd2ef38'))
(1276503626, BitArray('0x4ae6154c'))
(4009498663, BitArray('0x2718fcee'))
(2195688050, BitArray('0x728adf82'))
(859824952, BitArray('0x38e33f33'))
(2436222145, BitArray('0xc1cc3591'))
(2667970256, BitArray('0xd0fe059f'))
(3372662081, BitArray('0x41bd06c9'))
(2738999304, BitArray('0x08d041a3'))
(4235264195, BitArray('0xc30071fc'))
(3504191285, BitArray('0x35b7ddd0'))
(2158783448, BitArray('0xd86bac80'))
(3706267353, BitArray('0xd926e9dc'))
(2020726732, BitArray('0xccd77178'))
(1637089085, BitArray('0x3dff9361'))
(825020608, BitArray('0xc0d02c31'))
(1454531241, BitArray('0xa962b256'))
(4056381695, BitArray('0xff78c7f1'))
(2076483287, BitArray('0xd79ec47b'))


Best case c,k = 0,0; takes: 0.0s For 20 images: 0.5s  
Worst case c,k = 369,169; takes: 1m 24s For 20 images: 24m 12.4s  

In other words best case O(1), worst case: O(n^2)  
Average case: O(n^2)?  
  
Because of this generate a entropies based on only the first 841 frame sizes. (1-29 x 1-29)  

With the proposed change to the chatoization algorithm, the runtimes for 20 images for both cases takes approximately 0.4-0.6 seconds.